In [3]:
from google.cloud import bigquery as bq
import os
import pandas as pd
import pandas_gbq as pgbq
import util_functions as util

service_credentials = 'Service_Credentials/big-query-horse-play-f37757d450b8.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

bq_client = bq.Client()

In [5]:
sql = """
    SELECT
      CONCAT(
        'https://stackoverflow.com/questions/',
        CAST(id as STRING)) as url,
      view_count
    FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE tags like '%google-bigquery%'
    ORDER BY view_count DESC
    LIMIT 10
    """

In [6]:
# Defines the job to run
query_job = bq_client.query(sql)

# Runs it and returns the results
results = query_job.result()

In [7]:
# Need to iterate through the table results
for row in results:
    print(row)

Row(('https://stackoverflow.com/questions/13530967', 44081), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/13221978', 31409), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/22879669', 29177), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/6607552', 26722), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/16609219', 23515), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/22004216', 23226), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/35159967', 21839), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/10604135', 21751), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/10644993', 21110), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/13468933', 16062), {'url': 0, 'view_count': 1})


In [11]:
# This is where pandas comes in handy
results_df = pd.read_gbq(sql, dialect='standard')

In [12]:
results_df

,url,view_count
0,https://stackoverflow.com/questions/13530967,44081
1,https://stackoverflow.com/questions/13221978,31409
2,https://stackoverflow.com/questions/22879669,29177
3,https://stackoverflow.com/questions/6607552,26722
4,https://stackoverflow.com/questions/16609219,23515
5,https://stackoverflow.com/questions/22004216,23226
6,https://stackoverflow.com/questions/35159967,21839
7,https://stackoverflow.com/questions/10604135,21751
8,https://stackoverflow.com/questions/10644993,21110
9,https://stackoverflow.com/questions/13468933,16062


In [15]:
# Okay, going to try loading this in

dataset_ids = util.get_dataset_ids(bq_client)
dataset = util.get_dataset(bq_client, dataset_ids[1])

Datasets in project - big-query-horse-play:
names_dataset
second_test
---------------------------------------
Dataset ID: second_test
Friendly Name: None
Full ID: big-query-horse-play:second_test
Labels: {}
Project: big-query-horse-play
Ref: DatasetReference('big-query-horse-play', 'second_test')
---------------------------------------


In [19]:
results_df.to_gbq('second_test.sample_table', bq_client.project)
# Woo!


In [20]:
util.get_table_ids(bq_client, dataset)

Tables in second_test:
sample_table


['sample_table']

In [25]:
# Mmkay let's pull it back down from that source
sample_table = pd.read_gbq('SELECT * FROM `big-query-horse-play.second_test.sample_table`', dialect='standard')

In [26]:
sample_table

,url,view_count
0,https://stackoverflow.com/questions/13530967,44081
1,https://stackoverflow.com/questions/13221978,31409
2,https://stackoverflow.com/questions/22879669,29177
3,https://stackoverflow.com/questions/6607552,26722
4,https://stackoverflow.com/questions/16609219,23515
5,https://stackoverflow.com/questions/22004216,23226
6,https://stackoverflow.com/questions/35159967,21839
7,https://stackoverflow.com/questions/10604135,21751
8,https://stackoverflow.com/questions/10644993,21110
9,https://stackoverflow.com/questions/13468933,16062
